# Adaptive Active Learning: Example Workflow of `adaptive_al`

In [1]:
import torch
from pathlib import Path
import json

from adaptive_al.config import ExperimentConfig
from adaptive_al import ActiveLearning

import logging
logging.basicConfig(level=logging.INFO)

# Defining the Experiment Configuration

We begin by setting up the `ExperimentConfig`, which defines parameters like seed, number of active learning rounds, model setup, and storage directory. This configuration serves as the backbone of the experiment pipeline.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
cfg = ExperimentConfig(
    seed=42,
    total_rounds=5,
    experiment_name="dummy_test_pipeline",
    save_dir=Path("./experiments"),

    # Pool settings
    initial_pool_size=200,
    acquisition_batch_size=256,

    # Model
    model_name_or_path="distilbert-base-uncased",
    tokenizer_kwargs={
        "max_length": 128,
        "padding": "max_length",
        "truncation": True,
        "add_special_tokens": True,
        "return_tensors": "pt"
    },

    # Dataset names (for reference)
    data="agnews",
    num_labels=4,

    # Strategy
    strategy_class="DeltaF1Strategy",
    strategy_kwargs={"epsilon": 0.01, "k": 2}, # The base params are passed internally, only strategy specific params needed here

    optimizer_class = "Adam",
    optimizer_kwargs = {"lr": 1e-3, "weight_decay": 1e-4},

    criterion_class = "CrossEntropyLoss",
    criterion_kwargs = {},

    scheduler_class = "StepLR",
    scheduler_kwargs = {"step_size": 10, "gamma": 0.1},

    # Sampler
    sampler_class="RandomSampler",
    sampler_kwargs={"seed": 42},
    # sampler_class="EntropySampler",
    # sampler_kwargs={"show_progress": True},

    # Training
    device=device,
    epochs=3,
    batch_size=64
)

In [4]:
al = ActiveLearning(cfg)

INFO:root:Loading tokenizer and model from 'distilbert-base-uncased'...
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Train size: 118800, Validation size: 1200, Test size: 7600


# Running Active Learning Rounds

We now initialize the ActiveLearning object and step through multiple training rounds. Each round involves training on the current labeled pool, sampling new data, and updating model performance metrics.

In [5]:
print(f"Initial pool stats: {al.pool.get_pool_stats()}")

Initial pool stats: {'labeled_count': 200, 'unlabeled_count': 118600, 'total_count': 118800}


In [6]:
round_stats = al.train_one_round(new_indices=None)
print(f"Round 1 completed. Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")

INFO:root:
--- Round 1
INFO:root:Resetting model to initial state . . .
INFO:root:Epoch 1/3 completed in 2.16s | Avg Loss: 1.3966
INFO:root:Epoch 2/3 completed in 1.81s | Avg Loss: 1.5821
INFO:root:Epoch 3/3 completed in 2.03s | Avg Loss: 1.4010
INFO:root:Training completed in 6.00s
INFO:root:Model evaluation took 5.94 seconds
INFO:root:Round 1 complete. Val Stats: Loss=1.3925959436517013, F1=0.1, Time=6.03s


Round 1 completed. Val F1: 0.1000, Training Time: 6.03s


In [7]:
new_indices = al.sample_next_batch()
print(f"Sampled {len(new_indices)} new indices: {new_indices[:5]} ...")

INFO:root:Sampled 256 new samples in 0.0 seconds, using RandomSampler


Sampled 256 new indices: [83952, 14619, 3281, 97364, 36116] ...


In [8]:
round_stats = al.train_one_round(new_indices=new_indices)
print(f"Round 2 completed. Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")

INFO:root:
--- Round 2
INFO:root:Training with 256 new samples
INFO:root:Resetting model to initial state . . .
INFO:root:Epoch 1/3 completed in 1.83s | Avg Loss: 1.4489
INFO:root:Epoch 2/3 completed in 1.82s | Avg Loss: 1.1237
INFO:root:Epoch 3/3 completed in 1.81s | Avg Loss: 1.1825
INFO:root:Training completed in 5.47s
INFO:root:Model evaluation took 5.43 seconds
INFO:root:Round 2 complete. Val Stats: Loss=1.2624016184555857, F1=0.3203836511078514, Time=5.50s


Round 2 completed. Val F1: 0.3204, Training Time: 5.50s


In [9]:
num_additional_rounds = 3
for r in range(num_additional_rounds):
    print(f"\n--- Round {al.current_round + 1}")

    new_indices = al.sample_next_batch()
    if not new_indices:
        print("No more unlabeled data available!")
        break

    round_stats = al.train_one_round(new_indices=new_indices)
    print(f"Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")
    print(f"Pool Stats: {round_stats['pool_stats']}")

INFO:root:Sampled 256 new samples in 0.0 seconds, using RandomSampler
INFO:root:
--- Round 3
INFO:root:Training with 256 new samples
INFO:root:Resetting model to initial state . . .



--- Round 3


INFO:root:Epoch 1/3 completed in 1.79s | Avg Loss: 1.5235
INFO:root:Epoch 2/3 completed in 1.80s | Avg Loss: 1.5027
INFO:root:Epoch 3/3 completed in 1.80s | Avg Loss: 1.4264
INFO:root:Training completed in 5.38s
INFO:root:Model evaluation took 5.48 seconds
INFO:root:Round 3 complete. Val Stats: Loss=1.4086709712681018, F1=0.1, Time=5.41s
INFO:root:Sampled 256 new samples in 0.0 seconds, using RandomSampler
INFO:root:
--- Round 4
INFO:root:Training with 256 new samples
INFO:root:Resetting model to initial state . . .


Val F1: 0.1000, Training Time: 5.41s
Pool Stats: {'labeled_count': 200, 'unlabeled_count': 118600, 'total_count': 118800}

--- Round 4


INFO:root:Epoch 1/3 completed in 1.91s | Avg Loss: 1.5286
INFO:root:Epoch 2/3 completed in 2.08s | Avg Loss: 1.5585
INFO:root:Epoch 3/3 completed in 2.06s | Avg Loss: 1.0713
INFO:root:Training completed in 6.06s
INFO:root:Model evaluation took 6.07 seconds
INFO:root:Round 4 complete. Val Stats: Loss=0.8672537113490858, F1=0.5317818497627801, Time=6.08s
INFO:root:Sampled 256 new samples in 0.0 seconds, using RandomSampler
INFO:root:
--- Round 5
INFO:root:Training with 256 new samples
INFO:root:Resetting model to initial state . . .


Val F1: 0.5318, Training Time: 6.08s
Pool Stats: {'labeled_count': 200, 'unlabeled_count': 118600, 'total_count': 118800}

--- Round 5


INFO:root:Epoch 1/3 completed in 1.88s | Avg Loss: 1.5480
INFO:root:Epoch 2/3 completed in 1.99s | Avg Loss: 1.5503
INFO:root:Epoch 3/3 completed in 1.82s | Avg Loss: 1.1745
INFO:root:Training completed in 5.69s
INFO:root:Model evaluation took 5.42 seconds
INFO:root:Round 5 complete. Val Stats: Loss=2.763978406002647, F1=0.025287520197699836, Time=5.72s


Val F1: 0.0253, Training Time: 5.72s
Pool Stats: {'labeled_count': 200, 'unlabeled_count': 118600, 'total_count': 118800}


# End-to-End Active Learning Pipeline

Instead of manually stepping through rounds, we can run the entire active learning cycle using `run_full_pipeline()`. This method orchestrates all stages from training to evaluation automatically.

In [10]:
final_metrics = al.run_full_pipeline()
print(f"Final Test Metrics: F1={final_metrics['f1_score']:.4f}, Accuracy={final_metrics['accuracy']:.4f}, Loss={final_metrics['loss']:.4f}")

INFO:root:Loading tokenizer and model from 'distilbert-base-uncased'...
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Train size: 118800, Validation size: 1200, Test size: 7600
INFO:root:Running 5 rounds.
INFO:root:
--- Round 1
INFO:root:Resetting model to initial state . . .
INFO:root:Epoch 1/3 completed in 2.02s | Avg Loss: 1.3966
INFO:root:Epoch 2/3 completed in 1.95s | Avg Loss: 1.5821
INFO:root:Epoch 3/3 completed in 1.88s | Avg Loss: 1.4010
INFO:root:Training completed in 5.85s
INFO:root:Model evaluation took 5.52 seconds
INFO:root:Round 1 complete. Val Stats: Loss=1.3925959436517013, F1=0.1, Time=5.88s
INFO:root:Sampled 256 new samples in 0.0 seconds, using RandomSamp

Final Test Metrics: F1=0.0244, Accuracy=0.0389, Loss=2.7620


In [11]:
al.save_experiment()

INFO:root:Experiment saved to experiments\dummy_test_pipeline\results_20251002_231359.json


# Saving and Inspecting Experiment Results

After training, experiment results (metrics, configurations, and logs) are saved in JSON format. We can load these results for analysis, reproducibility, or comparison across different experiment runs.

In [12]:
with open(r"experiments/dummy_test_pipeline/results_20251002_231359.json", 'r') as f:
    experiment_data = json.load(f)

In [13]:
print(experiment_data.keys())

dict_keys(['cfg', 'total_rounds', 'round_val_stats', 'final_pool_stats', 'final_test_stats', 'confusion_matrix'])


In [14]:
experiment_data['cfg']

{'seed': 42,
 'total_rounds': 5,
 'initial_pool_size': 200,
 'acquisition_batch_size': 256,
 'min_rounds_before_plateau': -1,
 'plateau_patience': -1,
 'plateau_f1_threshold': 0.5,
 'approximate_evaluation_subset_size': -1,
 'max_seconds': None,
 'pool_proportion_threshold': -1,
 'sampler_class': 'RandomSampler',
 'sampler_kwargs': {'seed': 42},
 'strategy_class': 'DeltaF1Strategy',
 'strategy_kwargs': {'epsilon': 0.01, 'k': 2},
 'model_name_or_path': 'distilbert-base-uncased',
 'num_labels': 4,
 'tokenizer_kwargs': {'max_length': 128,
  'padding': 'max_length',
  'truncation': True,
  'add_special_tokens': True,
  'return_tensors': 'pt'},
 'optimizer_class': 'Adam',
 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 0.0001},
 'criterion_class': 'CrossEntropyLoss',
 'criterion_kwargs': {},
 'scheduler_class': 'StepLR',
 'scheduler_kwargs': {'step_size': 10, 'gamma': 0.1},
 'device': 'cuda',
 'epochs': 3,
 'batch_size': 64,
 'data': 'agnews',
 'save_dir': 'experiments',
 'experiment_nam

In [15]:
experiment_data['total_rounds']

5

In [16]:
experiment_data['round_val_stats'][-1]

{'training_time': 5.591256141662598,
 'avg_loss': 1.4242725372314453,
 'epochs': 3,
 'total_samples': 200,
 'new_samples': 0,
 'loss': 2.763978406002647,
 'f1_score': 0.025287520197699836,
 'accuracy': 0.04,
 'pool_stats': {'labeled_count': 200,
  'unlabeled_count': 118600,
  'total_count': 118800}}

In [17]:
experiment_data["final_pool_stats"]

{'labeled_count': 200, 'unlabeled_count': 118600, 'total_count': 118800}

In [18]:
experiment_data["final_test_stats"]

{'loss': 2.7620302208331453,
 'f1_score': 0.02437107500880923,
 'accuracy': 0.03894736842105263}